In [1]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'Malgun Gothic'
pd.set_option('mode.chained_assignment',  None) # 경고를 끈다
pd.set_option('display.max_columns', None) # 모든 열 출력
plt.rcParams['axes.unicode_minus'] = False

In [2]:
taas = pd.read_csv('data/taas/taas_preprocessed.csv', encoding='cp949')
motor = pd.read_csv('data/motorcycle/motorcycle_region.csv', encoding='cp949')
population = pd.read_csv('data/population/population_region.csv', encoding='cp949')
weather = pd.read_csv('data/weather/weather_taas.csv', encoding='cp949')

In [3]:
taas.head(1)

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3


In [4]:
motor.head(1)

,시도명,시군구,2019년 1월,2019년 2월,2019년 3월,2019년 4월,2019년 5월,2019년 6월,2019년 7월,2019년 8월,2019년 9월,2019년 10월,2019년 11월,2019년 12월,2020년 1월,2020년 2월,2020년 3월,2020년 4월,2020년 5월,2020년 6월,2020년 7월,2020년 8월,2020년 9월,2020년 10월,2020년 11월,2020년 12월,2021년 1월,2021년 2월,2021년 3월,2021년 4월,2021년 5월,2021년 6월,2021년 7월,2021년 8월,2021년 9월,2021년 10월,2021년 11월,2021년 12월
0,서울특별시,강남구,16456,16439,16463,16461,16561,16637,16623,16589,16642,16691,16757,16806,16656,16530,16737,16632,16701,16961,17186,17344,17687,17752,17776,17910,17898,17902,17872,17828,17695,17695,16643,16626,16513,16490,16162,16269


In [5]:
population.head(1)

,시도명,시군구,2019년 1월,2019년 2월,2019년 3월,2019년 4월,2019년 5월,2019년 6월,2019년 7월,2019년 8월,2019년 9월,2019년 10월,2019년 11월,2019년 12월,2020년 1월,2020년 2월,2020년 3월,2020년 4월,2020년 5월,2020년 6월,2020년 7월,2020년 8월,2020년 9월,2020년 10월,2020년 11월,2020년 12월,2021년 1월,2021년 2월,2021년 3월,2021년 4월,2021년 5월,2021년 6월,2021년 7월,2021년 8월,2021년 9월,2021년 10월,2021년 11월,2021년 12월
0,서울특별시,종로구,152866,152880,152778,152495,152220,152026,151858,151766,151767,151541,151457,151290,151215,151309,151217,150936,150623,150383,150183,149962,149952,149703,149549,149384,149125,148884,147296,147113,146377,146029,145692,145551,145512,145346,145073,144683


In [6]:
weather.head(1)

,사고번호,기온,강수량,풍속
0,2019010100100002,-5.5,0.0,1.0


### 전처리 함수 생성

In [7]:
def preprocess_func(df):
    # 데이터프레임 컬럼 이름을 list로 변환
    cols = list(df.columns[2:])

    # 새로운 컬럼 이름 리스트 생성
    new_cols = []
    for col in cols:
        year = col.split("년")[0]
        month = col.split("년")[1].split("월")[0].strip()
        if len(month) == 1:
            month = "0" + month
        new_col = year + month
        new_cols.append(new_col)
    a = list(df.columns[:2])
    df.columns = a + new_cols
    
    df['위치'] = df['시도명'] + ' ' + df['시군구']
    df = df.drop(['시도명', '시군구'], axis=1)
    df = df[['위치'] + [col for col in df.columns if col != '위치']]

    return df

In [8]:
motor = preprocess_func(motor)
population = preprocess_func(population)

In [9]:
motor.head(1)

,위치,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
0,서울특별시 강남구,16456,16439,16463,16461,16561,16637,16623,16589,16642,16691,16757,16806,16656,16530,16737,16632,16701,16961,17186,17344,17687,17752,17776,17910,17898,17902,17872,17828,17695,17695,16643,16626,16513,16490,16162,16269


In [10]:
population.head(1)

,위치,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
0,서울특별시 종로구,152866,152880,152778,152495,152220,152026,151858,151766,151767,151541,151457,151290,151215,151309,151217,150936,150623,150383,150183,149962,149952,149703,149549,149384,149125,148884,147296,147113,146377,146029,145692,145551,145512,145346,145073,144683


### taas 추가 전처리

In [11]:
taas['연월'] = taas['사고일시'].apply(lambda x: str(x)[:6])

In [12]:
taas['위치'] = taas['시군구'].apply(lambda x: ' '.join(x.split()[:2]))

In [13]:
taas['위치'] = taas['위치'].apply(lambda x: '세종특별자치시 세종특별자치시' if '세종특별자치시' in x else x)

In [14]:
taas['위치'].value_counts()

서울특별시 강남구    2354
대구광역시 달서구    2342
충청남도 천안시     1818
대구광역시 북구     1559
서울특별시 송파구    1503
             ... 
충청남도 청양군       78
인천광역시 강화군      71
인천광역시 동구       44
충청남도 계룡시       13
인천광역시 옹진군       8
Name: 위치, Length: 90, dtype: int64

### taas와 motor 합치기

In [15]:
taas_motor = pd.merge(taas, motor, how='outer',on='위치')
taas_motor.head(1)

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3,201901,서울특별시 구로구,15623,15624,15651,15678,15746,15763,15823,15868,15916,15956,15931,15467,15480,15456,15501,15509,15539,15566,15574,15639,15720,15749,15750,15693,15687,15662,15669,15616,15664,15664,15778,15073,14953,15024,14703,14691


In [16]:
for i in range(taas_motor.shape[0]):
    yearmonth = []
    yearmonth.append(taas_motor.iloc[i]['연월'])
    cols = taas_motor.columns.tolist()
    cols_to_keep = [x for x in cols if x not in yearmonth]
    taas_motor.loc[i,cols_to_keep] = 0

In [17]:
taas_motor

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61409,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122
61410,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122
61411,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122
61412,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122


In [18]:
taas_motor['이륜차'] = taas_motor.select_dtypes(include=["int"]).sum(axis=1)

In [19]:
taas_motor['이륜차'].isnull().sum()

0

In [20]:
taas_motor

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,이륜차
0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15623
1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15623
2,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15623
3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15623
4,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,15623,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61409,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122,7122
61410,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122,7122
61411,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122,7122
61412,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7122,7122


In [21]:
motor_sum = taas_motor['이륜차']

In [22]:
taas_1 = pd.concat([taas, pd.DataFrame(motor_sum)], axis=1)

In [23]:
taas_1.head(1)

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3,201901,서울특별시 구로구,15623


### taas와 population 합치기

In [24]:
taas_population = pd.merge(taas_1, population, how='outer',on='위치')
taas_population.head(1)

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3,201901,서울특별시 구로구,15623,404049,403668,404726,405516,405348,405415,405757,406359,406748,406889,407090,406664,406751,406245,405837,405271,405030,405075,405260,405495,405579,405481,405194,404408,403518,402642,401074,400399,399950,399266,398744,397887,397506,397330,397058,396754


In [25]:
for i in range(taas_population.shape[0]):
    yearmonth = []
    yearmonth.append(taas_population.iloc[i]['연월'])
    cols = taas_population.columns.tolist()
    cols_to_keep = [x for x in cols if x not in yearmonth]
    taas_population.loc[i,cols_to_keep] = 0

In [26]:
taas_population

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112
0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,404049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,404049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,404049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,404049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,404049,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61409,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222059
61410,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222059
61411,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222059
61412,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222059


In [27]:
taas_population['인구'] = taas_population.select_dtypes(include=["int"]).sum(axis=1)

In [28]:
taas_population['인구'].isnull().sum()

0

In [29]:
population_sum = taas_population['인구']

In [30]:
taas_2 = pd.concat([taas_1, pd.DataFrame(population_sum)], axis=1)

In [31]:
taas_2.head(1)

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,인구
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3,201901,서울특별시 구로구,15623,404049


### taas와 weather 합치기

In [32]:
weather.isnull().sum()

사고번호    0
기온      0
강수량     0
풍속      0
dtype: int64

In [33]:
taas_3 = taas_2.merge(weather, how='left',on='사고번호')

In [34]:
taas_3.isnull().sum()

사고번호        0
사고일시        0
요일          0
시군구         0
사고유형        0
법규위반        0
노면상태        0
기상상태        0
도로형태        0
가해운전자 차종    0
가해운전자 성별    0
가해운전자 연령    0
x           0
y           0
EPDO        0
연월          0
위치          0
이륜차         0
인구          0
기온          0
강수량         0
풍속          0
dtype: int64

In [35]:
taas_3.head(5)

,사고번호,사고일시,요일,시군구,사고유형,법규위반,노면상태,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,x,y,EPDO,연월,위치,이륜차,인구,기온,강수량,풍속
0,2019010100100002,201901010000,화요일,서울특별시 구로구 고척동,차대차,안전운전불이행,건조,맑음,단일로 - 기타,이륜,남,청년,126.867286,37.499889,3,201901,서울특별시 구로구,15623,404049,-5.5,0.0,1.0
1,2019010100100141,201901011100,화요일,서울특별시 서초구 서초동,차대차,안전거리미확보,건조,맑음,단일로 - 기타,승용,남,중년,127.017439,37.481949,3,201901,서울특별시 서초구,15623,404049,-3.7,0.0,2.5
2,2019010100100170,201901011300,화요일,서울특별시 서대문구 북아현동,차대사람,보행자보호의무위반,건조,맑음,단일로 - 기타,이륜,남,청년,126.956285,37.558028,3,201901,서울특별시 서대문구,15623,404049,-1.5,0.0,2.7
3,2019010100100236,201901011700,화요일,서울특별시 중랑구 상봉동,차대차,신호위반,건조,맑음,교차로 - 교차로안,이륜,남,청년,127.085745,37.596356,3,201901,서울특별시 중랑구,15623,404049,-1.8,0.0,2.2
4,2019010100100249,201901011800,화요일,서울특별시 구로구 천왕동,차대차,안전운전불이행,젖음/습기,맑음,교차로 - 교차로안,이륜,남,청년,126.842479,37.484115,3,201901,서울특별시 구로구,15623,404049,-2.7,0.0,2.6


In [36]:
# 합쳐진 taas 내보내기
taas_3.to_csv('data/taas/taas_motor_population_weather.csv', index=False, encoding = 'cp949')